In [60]:
import pandas as pd
import numpy as np

import time
import datetime as dt
from dateutil.relativedelta import relativedelta

import pprint

import seaborn as sns
import matplotlib.pyplot as plt

# Finance packages imports
import yfinance as yf
from sec_edgar_downloader import Downloader

# Web scraping imports
from bs4 import BeautifulSoup, SoupStrainer
import requests

from selenium import webdriver

import selenium as selenium
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

from IPython.display import clear_output

date_now = dt.date.today()

## QOL Functions

In [61]:
def Safe_clear():
    input("Press enter to continue...")
    clear_output(wait = True)

In [62]:
def Option_input(num_options: int = 0) -> int:

    if num_options == 0:
        while True:
            option = input().strip().lower()
            if option == "yes":
                return 1

            elif option == "no":
                return 0

            else:
                print("Please enter: \"yes\" or \"no\"")
                continue      

    else:
        while True:
            try:
                option = int(input("Please enter the list number only: "))
                if option <= num_options:
                    return option
        
                else:
                    print("Please enter a valid number from 1 to ", num_options)
                    continue
    
            except ValueError:
                print("Please enter a valid number from 1 to ", num_options)
                continue

## 1. Risk Free Rate

In [63]:
def Risk_free_rate( ticker: str) -> float:
    yf_obj = yf.Ticker(ticker)
    current_yield = yf_obj.history(period="1d")["Close"].iloc[-1]
    risk_free_rate = current_yield/100
    return risk_free_rate

## 2. Market Premiums

### Mature Market Premium (USA)

Implied Equity Risk Premium is a forward looking measure and can be updated daily on easily available data. But, this data is not available to me in any meaningful and tangible form and therefore, trades will be done on an MMP at year - 1

In [6]:
# Calculating the implied equity risk premiums using the S&P 500
#    this serves as the baseline equity risk premium on top of which,
#    further regional level risks can be added

In [65]:
def Mature_market_premium( type: str = "fcfe" ) -> float:

    implied_erp = pd.read_excel("data/histimpl.xlsx", sheet_name = "Historical Impl Premiums", skiprows = 6)[:65]
    ierp_ddm = implied_erp.loc[ implied_erp["Year"].astype("Int64") == date_now.year - 1]["Implied Premium (DDM)"].iloc[0]
    ierp_fcfe = implied_erp.loc[ implied_erp["Year"].astype("Int64") == date_now.year - 1]["Implied ERP (FCFE)"].iloc[0]
    ierp_fcfe_sustainable_payout = implied_erp.loc[ implied_erp["Year"].astype("Int64") == 2024]["Implied Premium (FCFE with sustainable Payout)"].iloc[0]

    if type.lower() == "ddm":
        return ierp_ddm

    elif type.lower() == "fcfe":
        return ierp_fcfe

    elif "fcfe" in type.lower() and "sustain" in type.lower():
        return ierp_fcfe_sustainable_payout

## 3. Country Risk Premium

### 3.1 Relative Equity Volatility - Emerging Markets IEMG / EMB up to date

Relative Equity Volatility is being used here to scale the Country Risk Premium (derived from CDS spread) of the focal company.

Relative Volatility is measured as:

Mean of Annualised Volatility over last 5 years of a Gov Bonds ETF / 
Mean of Annualised Volatility over last 5 years of a Market Equity ETF

Annualised Volatility = std(daily Adj Close of ETF) *  ((no. of trading years) ** 0.5)

In [4]:
def Relative_volatility_uptodate() -> float:
    
    # iShares JP Morgan USD Emerging Markets Bond ETF
    emb_adj_close = yf.download("EMB", end = date_now,
                                start = date_now - relativedelta(years = 5) , auto_adjust = True)
    
    # this is not percentage, but fractional, multiply by 100 if you want the percent change
    emb_adj_close_pct_change = emb_adj_close.loc[:, ("Close", "EMB")].pct_change() * 100

    # iShares Core MSCI Emerging Markets ETF
    iemg_adj_close = yf.download("IEMG", end = date_now,
                                 start = date_now - relativedelta(years = 5) , auto_adjust = True)
    
    # this is not percentage, but fractional, multiply by 100 if you want the percent change
    iemg_adj_close_pct_change = iemg_adj_close.loc[:, ("Close", "IEMG")].pct_change() * 100

    iemg_annual_volatility = []
    emb_annual_volatility = []
    
    # the slice object is used to measure annualised volatility  
    #                                     = std(Daily data in a year) * root (no. of trading days in the year)
    
    for i in range(0, 5):
        iemg_slice = iemg_adj_close_pct_change.to_frame().loc[date_now - relativedelta(years = 5 - i): date_now - relativedelta(years = 4 - i)]
        emb_slice = emb_adj_close_pct_change.to_frame().loc[date_now - relativedelta(years = 5 - i): date_now - relativedelta(years = 4 - i)]
        
        iemg_slice_annualised_vol = iemg_slice.std().values[0] * (len(iemg_slice)**0.5)
        emb_slice_annualised_vol = emb_slice.std().values[0] * (len(emb_slice)**0.5)
    
        iemg_annual_volatility.append(iemg_slice_annualised_vol)
        emb_annual_volatility.append(emb_slice_annualised_vol)
    
    annual_rel_volatility = [i/s for i,s in zip(iemg_annual_volatility,emb_annual_volatility)]
    relative_volatility = np.mean(annual_rel_volatility)
    clear_output(wait = True)
    
    return relative_volatility
    
    

### 3.2 CDS/PRS Spread proxy for CRP

In [38]:
def Country_risk_premium( country: str ) -> float:
    
    crp_data = pd.read_excel("Country Premiums.xlsx", sheet_name = "ERPs by country", skiprows = 6)
    crp = crp_data.loc[crp_data["Country"] == country, "Country Risk Premium"].values[0]
    
    while True:        
        print("Do you want to use Relative Equity Volatility to scale CRP? Yes/No")
        prompt1 = Option_input()
        
        if prompt1 == 1:
            relative_volatility_uptodate = Relative_volatility_uptodate()
            
            print("Which measure do you want to use?\n1. Relative Volatility = 1.35 : as calculated by Aswath D. by comparing SCRTEM / EMB at the start of the year 2025")
            print("2. Relative Volatility =", relative_volatility_uptodate,": Up to date measure by comparing IEMG / EMB")
            print("3. Input your own Relative Volatility measure")
            option = Option_input(3)

            if option == 1:
                relative_volatility_selected = 1.35
            elif option == 2:
                relative_volatility_selected = relative_volatility_uptodate
            elif option == 3:
                while True:
                    try:
                        relative_volatility_selected = float(input("Custom Relative Volatility: "))
                        break
                    except ValueError:
                        print("Enter a valid float value.")
                        continue

            print("CRP of", country, "=", crp)
            print("You have selected -- > Relative Volatility =", relative_volatility_selected)
            print("CRP scaled by relative volatility =", crp*relative_volatility_selected)
            print("Yes to Confirm, No to choose again\n")
            
            prompt2 = Option_input()
            if prompt2 == 1:
                clear_output(wait = True)
                return crp*relative_volatility_selected              

        if prompt1 == 0:
            relative_volatility_selected = 1
            print("CRP of", country, "=", crp)
            print("You have selected -- > Relative Volatility =", relative_volatility_selected, "[1 = unscaled]")
            print("Unscaled CRP =", crp*relative_volatility_selected)
            print("Yes to Confirm, No to choose again\n")
            prompt2 = input()
            if prompt2.lower() == "yes":
                clear_output(wait = True)
                return crp*relative_volatility_selected

        clear_output(wait = True)
        
    

## 4. Tickers Scraping and YF obj creation

In [52]:
def US_FinViz_scrape() -> list[str]:
    
    driver = webdriver.Chrome()
    url = "https://finviz.com/screener.ashx"
    driver.get(url)
    
    # filtering in the page
    industry_filter = Select(driver.find_element(By.ID, "fs_ind"))   
    country_filter = Select(driver.find_element(By.ID, "fs_geo"))

    while True:

        clear_output(wait = True)
        print("Country of trade: USA\nWhich of the following industries do you wish to focus on? :")
        for i, opt in enumerate(industry_filter.options, 1):
            print(f"{i}. {opt.text}")

        industry = list(enumerate(industry_filter.options, 1))[Option_input(len(industry_filter.options)) - 1][1].text

        clear_output(wait = True)
        print("Country of trade: USA\nCountry of origin for your companies?")
        for i, opt in enumerate(country_filter.options, 1):
            print(f"{i}. {opt.text}")
        
        origin_country = list(enumerate(country_filter.options))[Option_input(len(industry_filter.options)) - 1][1].text
  
        clear_output(wait = True)
        print("Confirm the following selection:\n Industry: ", industry, "\n Country of origin: ", origin_country)
        print("\nYes to confirm / No to choose again")
        option = Option_input()
        if option == 1:
            print("Please wait.....")
            break

    clear_output(wait = True)


    industry_filter = Select(driver.find_element(By.ID, "fs_ind"))
    industry_filter.select_by_visible_text(industry)
    country_filter = Select(driver.find_element(By.ID, "fs_geo"))
    country_filter.select_by_visible_text(origin_country)
    
    # scraping ticker symbol in the pages under the filter:
    tickers = []
    pageSelect = Select(driver.find_element(By.ID, "pageSelect"))
    pages = pageSelect.options
    
    for i in range(len(pages)):
        pageSelect = Select(driver.find_element(By.ID, "pageSelect"))
        pageSelect.select_by_index(i)
        table = driver.find_element(By.CLASS_NAME, "styled-table-new")
        rows = table.find_elements(By.CLASS_NAME, "styled-row")
        for i in range(len(rows)):
            tickers.append(rows[i].find_elements(By.CSS_SELECTOR, "td")[1].text)
    
    driver.quit()
    
    return tickers

In [50]:
def Country_industry_tickers(country: str) -> list[str]:
    
    if (country == "United States"):
        tickers = US_FinViz_scrape()

    return tickers

    

In [ ]:
def Tickers_instantiation( tickers: list[str] ) -> :
    for ticker in tickers:
        

## Ticker Class

In [ ]:
class Ticker:
    def __init__(self, symbol, date_now):
        self.symbol = symbol.upper()
        self.yf_obj = yf.Ticker(self.symbol)
        
        self.start_date = date_now

## Input Parameters

In [14]:
rf_ticker = "^TNX"  # 10 year USTreasury bond
rf = Risk_free_rate(rf_ticker)
print(rf)

0.04493000030517578


In [55]:
country_options = ["United States"]

print("Available Countries:")
for i, country in enumerate(country_options, 1):
      print(f"{i}. {country}")

print("Select your country of trade [input option number only]: ")
country_index = Option_input(len(country_options))

Available Countries:
1. United States
Select your country of trade [input option number only]: 


Please enter the list number only:  1


In [ ]:
country_risk_premium = Country_risk_premium("United States")

In [56]:
lmt = yf.Ticker("LMT")

In [58]:
dividends = lmt.dividends

In [59]:
dividends

Date
1984-05-22 00:00:00-04:00    0.046012
1984-08-14 00:00:00-04:00    0.046012
1984-11-13 00:00:00-05:00    0.046012
1985-02-08 00:00:00-05:00    0.046012
1985-05-21 00:00:00-04:00    0.061350
                               ...   
2024-02-29 00:00:00-05:00    3.150000
2024-06-03 00:00:00-04:00    3.150000
2024-09-03 00:00:00-04:00    3.150000
2024-12-02 00:00:00-05:00    3.300000
2025-03-03 00:00:00-05:00    3.300000
Name: Dividends, Length: 164, dtype: float64

In [65]:
spy.analyst

AttributeError: 'Ticker' object has no attribute 'analyst'

In [62]:
spy.cashflow

""


In [66]:
spy.info

{'longBusinessSummary': 'The trust seeks to achieve its investment objective by holding a portfolio of the common stocks that are included in the index, with the weight of each stock in the portfolio substantially corresponding to the weight of such stock in the index.',
 'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 524.58,
 'open': 523.01,
 'dayLow': 520.1,
 'dayHigh': 536.43,
 'regularMarketPreviousClose': 524.58,
 'regularMarketOpen': 523.01,
 'regularMarketDayLow': 520.1,
 'regularMarketDayHigh': 536.43,
 'trailingPE': 23.76753,
 'volume': 97199285,
 'regularMarketVolume': 97199285,
 'averageVolume': 70574723,
 'averageVolume10days': 149427160,
 'averageDailyVolume10Day': 149427160,
 'bid': 534.1,
 'ask': 534.54,
 'bidSize': 8,
 'askSize': 22,
 'yield': 0.0128,
 'totalAssets': 576008880128,
 'fiftyTwoWeekLow': 481.8,
 'fiftyTwoWeekHigh': 613.23,
 'fiftyDayAverage': 574.5602,
 'twoHundredDayAverage': 573.9236,
 'trailingAnnualDivi

## Ticker instantiation

In [ ]:
dat = yf.Ticker("MSFT")

In [ ]:
pprint.pprint(india_yield.info)

In [ ]:
tnx = yf.Ticker("^TNX")
current_yield = tnx.history(period="1d")["Close"].iloc[-1]
risk_free_rate = current_yield / 100
print(f"10Y Risk-Free Rate: {risk_free_rate:.4%}")


In [ ]:
print(f"10Y Risk-Free Rate: {risk_free_rate:.4f}")


In [ ]:
pprint.pprint(tnx.info)

In [ ]:
dat.calendar


In [ ]:
dat.analyst_price_targets


In [ ]:
dat.quarterly_income_stmt


In [ ]:
dat.history(period='1mo')


In [ ]:
dat.option_chain(dat.options[0]).calls

In [ ]:
dat.options

In [ ]:
data = yf.download("MSFT", period="1000d")
print(data)